In [98]:
import os, glob, random
import pandas as pd

class External:
    def __init__(self, imgFolder, txtFolder, externalCsvPath, excludeFolder, classL):
        self.model        = ["FasterRCNN", "YoloR", "YOLOv7"][2]
        self.df           = pd.read_csv(externalCsvPath)[['file',f'{self.model}_single',f'{self.model}_missing']]
        self.df["file"]   = self.df["file"].apply(lambda s:s.split('\\')[-1])
        print( self.df )
        self.imgFolder    = imgFolder
        self.txtFolder    = txtFolder
        self.outputFolder = f"{os.path.dirname(externalCsvPath)}/data"
        self.excludeFolder= excludeFolder
        self.classL       = classL
        os.makedirs(self.outputFolder, exist_ok=True)
        
    def getError(self, num=100): # YOLOv7
        R = list(range(len(self.df)))
        random.Random(7).shuffle(R)
        self.df['r'] = R
        singleL    = self.df[ self.df[f'{self.model}_single']!=0  ].sort_values(by=[f'{self.model}_single', 'r'], ascending=False)['file'].to_list()[:num]
        missL      = self.df[ self.df[f'{self.model}_missing']!=0 ].sort_values(by=[f'{self.model}_missing','r'], ascending=False)['file'].to_list()[:num*2]
        excludeS   = { os.path.basename(path) for path in glob.glob(f"{self.excludeFolder}/*.jpg") }
        imgNameS   = {*singleL, *missL}.difference(excludeS)
        print( f"singleL[:3]={singleL[:2]}, missL[:3]={missL[:2]}")
        print( f"len(singleL)={len(singleL)}, len(missL)={len(missL)}, len(imgNameS)={len(imgNameS)}" ); #raise
        F          = lambda x:str(round(float(x),5))
        for imgName in imgNameS:
            os.system(f"cp {self.imgFolder}/{imgName} {self.outputFolder}")
            with open(f"{self.outputFolder}/{imgName.replace('.jpg','.txt')}","w") as f:
                for line in open(f"{self.txtFolder}/{imgName.replace('.jpg','.txt')}").readlines():
                    cid, cx, cy, w, h, conf = line[:-1].split(" ")
                    if float(conf)>=0.5:
                        f.write( " ".join([cid,F(cx),F(cy),F(w),F(h)])+"\n" )
        open(f"{self.outputFolder}/classes.txt", "w").write("\n".join(self.classL))

In [99]:
obj = External("/home/jovyan/data-vol-2/recycling/backboneData/2022-06-14/", "../runs/test/unlabeled0614_v4v2/yoloFormat/", "../_compare/0915_0614_0714/3_models_2022-06-14.csv",\
               "../_data/sampleV5/data", ['tetra'])
obj.getError()

                         file  YOLOv7_single  YOLOv7_missing
0     20220614_100019_684.jpg              0               0
1     20220614_100019_867.jpg              0               0
2     20220614_100020_066.jpg              0               0
3     20220614_100020_266.jpg              0               0
4     20220614_100020_466.jpg              0               0
...                       ...            ...             ...
3140  20220614_105846_268.jpg              0               0
3141  20220614_105846_466.jpg              0               0
3142  20220614_105846_668.jpg              0               0
3143  20220614_105846_867.jpg              0               0
3144  20220614_105847_066.jpg              0               0

[3145 rows x 3 columns]
singleL[:3]=['20220614_105718_069.jpg', '20220614_104449_670.jpg'], missL[:3]=['20220614_102208_668.jpg', '20220614_105823_868.jpg']
len(singleL)=100, len(missL)=47, len(imgNameS)=113


In [100]:
obj = External("/home/jovyan/data-vol-2/recycling/backboneData/2022-07-14/", "../runs/test/unlabeled0714_v4v2/yoloFormat/", "../_compare/0915_0614_0714/3_models_2022-07-14.csv",\
               "../_data/sampleV5/data")
obj.getError()

                         file  YOLOv7_single  YOLOv7_missing
0     20220714_160410_969.jpg              0               1
1     20220714_160411_169.jpg              0               0
2     20220714_160411_370.jpg              0               0
3     20220714_160411_579.jpg              1               0
4     20220714_160411_809.jpg              0               0
...                       ...            ...             ...
2167  20220714_165205_356.jpg              0               0
2168  20220714_165205_557.jpg              0               0
2169  20220714_165205_758.jpg              0               0
2170  20220714_165214_175.jpg              0               0
2171  20220714_165214_777.jpg              0               0

[2172 rows x 3 columns]
singleL[:3]=['20220714_160618_377.jpg', '20220714_165012_556.jpg'], missL[:3]=['20220714_160441_372.jpg', '20220714_164858_153.jpg']
len(singleL)=100, len(missL)=200, len(imgNameS)=271


In [101]:
!ls ../_compare/0915_0614_0714/data | wc -l

768
